<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [268]:
import gEconpy as ge
import sympy as sp
from gEconpy.shared.utilities import eq_to_ss

In [277]:
mod = ge.gEconModel('gcn_files/2_region.gcn')

Model Building Complete.
Found:
	42 equations
	42 variables
	The following "variables" were defined as constants and have been substituted away:
		B_t,mc_Y_t
	4 stochastic shocks
		 0 / 4 has a defined prior. 
	20 parameters
		 0 / 20 has a defined prior. 
	0 calibrating equations
	0 parameters to calibrate
 Model appears well defined and ready to proceed to solving.



In [285]:
mod.steady_state()

Steady state found! Sum of squared residuals is 1.8605349885125277e-29


In [288]:
mod.print_steady_state()

C_ss                         1.391
Div_A_ss                     0.299
Div_B_ss                     0.299
Div_Y_ss                     0.000
Div_ss                       0.599
I_ss                         0.263
K_d_A_ss                     5.261
K_d_B_ss                     5.261
K_ss                        10.523
LHS_A_ss                     1.675
LHS_B_ss                     1.675
L_d_A_ss                     0.330
L_d_B_ss                     0.330
L_ss                         0.660
P_A_ss                       1.000
P_A_star_ss                  0.985
P_B_ss                       1.000
P_B_star_ss                  0.985
P_nu_A_ss                    1.052
P_nu_B_ss                    1.052
RHS_A_ss                     1.047
RHS_B_ss                     1.047
TC_A_ss                     -0.528
TC_B_ss                     -0.528
TC_Y_ss                     -1.654
U_ss                       -86.060
Y_A_ss                       0.827
Y_B_ss                       0.827
Y_f_A_ss            

In [290]:
mod.solve_model()

Solution found, sum of squared residuals:  7.966819739692769e+26
Norm of deterministic part: 28225555335002.285156250
Norm of stochastic part:    28225555335002.285156250
